# ETL with movies datasets from IMDb

This Jupiter Notebook contains the following sections:

1. EXTRACT: Extract the data from de csv files
2. CLEANING AND TRANSFORMATION PROCESS: Nested Columns, Missing Data
3. LOADING: Load the data for future use

In [129]:
# Import all the libraries needed
import pandas as pd
import numpy as np
import ast
import json

# 1. EXTRACT

In [130]:
# Load the first dataset
mv = pd.read_csv('datasets/raw_data/movies_dataset.csv')
mv.head(1)

/var/folders/7b/x372r2jn4b95xj0_xs49ys4h0000gn/T/ipykernel_28649/735640281.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mv = pd.read_csv('datasets/raw_data/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [131]:
# Load the second dataset
credits = pd.read_csv('datasets/raw_data/credits.csv')
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


# 2. CLEANING AND TRANSFORMATION PROCESS

In [132]:
# Convert the type of Credits.id for merge
credits["id"] = credits["id"].astype(str)


In [133]:
mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [134]:
# Drop the unnecessary columns 
mv = mv.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])
mv.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0


In [135]:
# looking for NaN in belongs to collection
mv['belongs_to_collection'].isna().value_counts()

belongs_to_collection
True     40972
False     4494
Name: count, dtype: int64

In [136]:
# fill the NaN with {} values

mv['belongs_to_collection'].fillna('{}', inplace=True)

In [137]:
mv['belongs_to_collection'].head()

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                   {}
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                   {}
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [138]:
# new df with belongs to collection and mapping each row with eval
df_btc = mv['belongs_to_collection'].map(eval)

# Apply pd.Series to expand dataframe
df_btc = df_btc.apply(pd.Series)

In [139]:
# Rename the columns id and name and drop the unnecessary columns.
df_btc.rename(columns={'id': 'id_collection', 'name': 'collection'}, inplace=True)
df_btc.drop(columns=['poster_path', 'backdrop_path'], inplace=True)

In [140]:
df_btc.head()

,id_collection,collection,0
0,10194.0,Toy Story Collection,NaN
1,NaN,NaN,NaN
2,119050.0,Grumpy Old Men Collection,NaN
3,NaN,NaN,NaN
4,96871.0,Father of the Bride Collection,NaN


In [141]:
df_btc.drop(columns=0, inplace=True)

In [142]:
df_genres = pd.DataFrame()
df_genres['genres'] = mv['genres']

In [143]:
df_genres.head(1)

,genres
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."


In [144]:
# check and transform str to dict
df_genres['genres'] = df_genres['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
df_genres['genre_id'] = df_genres['genres'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
df_genres['genre'] = df_genres['genres'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [145]:
df_genres.head()

,genres,genre_id,genre
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[16, 35, 10751]","[Animation, Comedy, Family]"
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[12, 14, 10751]","[Adventure, Fantasy, Family]"
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[10749, 35]","[Romance, Comedy]"
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[35, 18, 10749]","[Comedy, Drama, Romance]"
4,"[{'id': 35, 'name': 'Comedy'}]",[35],[Comedy]


In [146]:
# drop nested column
df_genres.drop(columns='genres', inplace=True)

In [147]:
# Function to transform the columns
def clean_column_values(df, column_name):
    df[column_name] = df[column_name].astype(str).str.replace('[', '', regex=False)
    df[column_name] = df[column_name].astype(str).str.replace(']', '', regex=False)
    df[column_name] = df[column_name].astype(str).str.replace("'", '', regex=False)
    return df

In [148]:
df_genres.head()

,genre_id,genre
0,"[16, 35, 10751]","[Animation, Comedy, Family]"
1,"[12, 14, 10751]","[Adventure, Fantasy, Family]"
2,"[10749, 35]","[Romance, Comedy]"
3,"[35, 18, 10749]","[Comedy, Drama, Romance]"
4,[35],[Comedy]


In [149]:
# The int Dtype is object
mv['id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: id
Non-Null Count  Dtype 
--------------  ----- 
45466 non-null  object
dtypes: object(1)
memory usage: 355.3+ KB


In [150]:
mv['production_companies'].isna().value_counts()

production_companies
False    45463
True         3
Name: count, dtype: int64

In [151]:
mv['production_companies'].fillna('{}', inplace=True)

In [152]:
df_companies = pd.DataFrame()
df_companies['production_companies'] = mv['production_companies']

In [153]:
# check and transform str to dict
df_companies['production_companies'] = df_companies['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
df_companies['companies_id'] = df_companies['production_companies'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
df_companies['companies_name'] = df_companies['production_companies'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [154]:
df_companies.drop(columns='production_companies', inplace=True)

In [155]:
df_companies.head()

,companies_id,companies_name
0,[3],[Pixar Animation Studios]
1,"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co..."
2,"[6194, 19464]","[Warner Bros., Lancaster Gate]"
3,[306],[Twentieth Century Fox Film Corporation]
4,"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]"


In [156]:
mv['production_countries'].fillna('{}', inplace=True)

In [157]:
countries = pd.DataFrame()
countries['production_countries'] = mv['production_countries']

In [158]:
countries.head(1)

,production_countries
0,"[{'iso_3166_1': 'US', 'name': 'United States o..."


In [159]:

# check and transform str to dict
countries['production_countries'] = countries['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
countries['iso'] = countries['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x] if isinstance(x, list) else [])
countries['countrie_name'] = countries['production_countries'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])


In [160]:
countries.drop(columns='production_countries', inplace=True)

In [161]:
countries.head()

,iso,countrie_name
0,[US],[United States of America]
1,[US],[United States of America]
2,[US],[United States of America]
3,[US],[United States of America]
4,[US],[United States of America]


In [162]:
mv['spoken_languages'].fillna('{}', inplace=True)

In [163]:
lang = pd.DataFrame()
lang['spoken_languages'] = mv['spoken_languages']

In [164]:
lang.head(1)

,spoken_languages
0,"[{'iso_639_1': 'en', 'name': 'English'}]"


In [165]:
# check and transform str to dict
lang['spoken_languages'] = lang['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
lang['iso_lang'] = lang['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x] if isinstance(x, list) else [])
lang['lang_name'] = lang['spoken_languages'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [166]:
lang.drop(columns='spoken_languages', inplace=True)
lang.head()

,iso_lang,lang_name
0,[en],[English]
1,"[en, fr]","[English, Français]"
2,[en],[English]
3,[en],[English]
4,[en],[English]


In [167]:
# Concat the DFs with the unnested columns

mv = pd.concat([mv, df_btc, df_genres, df_companies, countries, lang], axis=1)

In [168]:
# drop unnecessary columns

mv.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace=True)

In [169]:
# Fill nan values in revenue and budget
mv['revenue'].fillna(0, inplace=True)
mv['budget'].fillna(0, inplace=True)

In [170]:
mv['release_date'].isna().value_counts()

release_date
False    45379
True        87
Name: count, dtype: int64

In [171]:
# convert to datetime format, coerce: non valid values as nan
mv['release_date'] = pd.to_datetime(mv['release_date'], errors='coerce')

In [172]:
# fromat the date as yy-mm-dd
mv['release_date'] = mv['release_date'].dt.strftime("%Y-%m-%d")

In [173]:
# Drop nan values in release date
mv.dropna(subset=['release_date'], inplace=True)

In [174]:
# create Release_year 
mv['release_year'] = pd.to_datetime(mv['release_date']).dt.year

In [175]:
# Format budget, if there is a str it will be formatted as nan
mv['budget'] = pd.to_numeric(mv['budget'], errors='coerce')


In [176]:
mv['budget'] = mv['budget'].astype(float)

In [177]:
mv['revenue'] = mv['revenue'].astype(float)

In [178]:
# Create revenue with np.where. If there is no data, then the output is 0
mv['return'] = np.where(mv['budget'] != 0, mv['revenue'] / mv['budget'], 0)

In [179]:
mv['return'].head()

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

In [180]:
mv['id'].duplicated().value_counts()

id
False    45346
True        30
Name: count, dtype: int64

In [181]:
mv.drop_duplicates(subset='id', inplace=True)

In [182]:
mv['id'] = mv['id'].astype(int)

In [183]:
mv['release_date'] = pd.to_datetime(mv['release_date'])
mv['release_date']

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45346, dtype: datetime64[ns]

In [184]:
mv.shape

(45346, 25)

# Credits DataSet

In [185]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [186]:
cast = pd.DataFrame()
cast['cast'] = credits['cast']

In [187]:
# check and transform str to dict
cast['cast'] = cast['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
cast['cast_id'] = cast['cast'].apply(lambda x: [d['cast_id'] for d in x] if isinstance(x, list) else [])
cast['character'] = cast['cast'].apply(lambda x: [d['character'] for d in x] if isinstance(x, list) else [])
cast['credit_id'] = cast['cast'].apply(lambda x: [d['credit_id'] for d in x] if isinstance(x, list) else [])
cast['actor_id'] = cast['cast'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
cast['actor_name'] = cast['cast'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [188]:
cast.head(1)

,cast,cast_id,character,credit_id,actor_id,actor_name
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[14, 15, 16, 17, 18, 19, 20, 26, 22, 23, 24, 2...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[52fe4284c3a36847f8024f95, 52fe4284c3a36847f80...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."


In [189]:
cast.drop(columns='cast', inplace=True)
cast.head()

,cast_id,character,credit_id,actor_id,actor_name
0,"[14, 15, 16, 17, 18, 19, 20, 26, 22, 23, 24, 2...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[52fe4284c3a36847f8024f95, 52fe4284c3a36847f80...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[1, 8, 2, 24, 10, 25, 26, 11, 14, 13, 31, 12, ...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[52fe44bfc3a36847f80a7c73, 52fe44bfc3a36847f80...","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[2, 3, 4, 5, 6, 9, 10]","[Max Goldman, John Gustafson, Ariel Gustafson,...","[52fe466a9251416c75077a8d, 52fe466a9251416c750...","[6837, 3151, 13567, 16757, 589, 16523, 7166]","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[1, 2, 3, 4, 5, 6, 8, 10, 20, 21]","[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[52fe44779251416c91011aad, 52fe44779251416c910...","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[1, 2, 3, 4, 13, 14, 15, 16, 17, 18, 19, 20]","[George Banks, Nina Banks, Franck Eggelhoffer,...","[52fe44959251416c75039eb9, 52fe44959251416c750...","[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [190]:
cast.drop(columns=['cast_id', 'credit_id'], inplace=True)

In [191]:
cast.head()

,character,actor_id,actor_name
0,"[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[Max Goldman, John Gustafson, Ariel Gustafson,...","[6837, 3151, 13567, 16757, 589, 16523, 7166]","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[George Banks, Nina Banks, Franck Eggelhoffer,...","[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [192]:
crew = pd.DataFrame()
crew['crew'] = credits['crew']

In [193]:
# check and transform str to dict
crew['crew'] = crew['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# create new columns from dict
crew['crew_id'] = crew['crew'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
crew['job'] = crew['crew'].apply(lambda x: [d['job'] for d in x] if isinstance(x, list) else [])
crew['crew_name'] = crew['crew'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [194]:
crew.drop(columns='crew', inplace=True)

In [195]:
crew.head()

,crew_id,job,crew_name
0,"[7879, 12891, 7, 12892, 12893, 12894, 12895, 1...","[Director, Screenplay, Screenplay, Screenplay,...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."
1,"[511, 876, 1729, 4945, 4951, 4952, 8023, 9967,...","[Executive Producer, Screenplay, Original Musi...","[Larry J. Franco, Jonathan Hensleigh, James Ho..."
2,"[26502, 16837, 16837, 1551320]","[Director, Characters, Writer, Sound Recordist]","[Howard Deutch, Mark Steven Johnson, Mark Stev..."
3,"[2178, 5144, 5144, 21968, 70592, 111118, 11111...","[Director, Screenplay, Producer, Producer, Pro...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez..."
4,"[37, 5506, 17698, 17698, 26160, 56106, 68755]","[Original Music Composer, Director of Photogra...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N..."


In [196]:
# function to extract director's name
def extract_director(row):
    job_list = row['job']
    name_list = row['crew_name']
    directors = []  
    for i in range(len(job_list)):
        if job_list[i] == 'Director':
            directors.append(name_list[i])
    return ', '.join(directors)  # return directors names separated by comma if there are more than one

# apply the function to dataframe
crew['Director'] = crew.apply(extract_director, axis=1)

# show the results
print(crew[['Director']])


               Director
0         John Lasseter
1          Joe Johnston
2         Howard Deutch
3       Forest Whitaker
4         Charles Shyer
...                 ...
45471  Hamid Nematollah
45472          Lav Diaz
45473    Mark L. Lester
45474  Yakov Protazanov
45475     Daisy Asquith

[45476 rows x 1 columns]


In [197]:
crew.drop(columns=['crew_id', 'job', 'crew_name'], inplace=True)

In [198]:
crew.head()

,Director
0,John Lasseter
1,Joe Johnston
2,Howard Deutch
3,Forest Whitaker
4,Charles Shyer


In [199]:
# Concat the DFs with the unnested columns

credits = pd.concat([credits, cast, crew], axis=1)

In [200]:
credits.drop(columns=['cast', 'crew'], inplace=True)

# 3. LOAD THE DATA

In [201]:
mv.to_csv('datasets/cleaned_data/movies_cleaned.csv', index=False)

In [ ]:
credits.to_csv('datasets/cleaned_data/credits_cleaned.csv', index=False)